In [2]:
import os
import json

def read_jsons_from_directory(directory_path):
    all_jsons = []
    # List all files in the given directory
    for filename in os.listdir(directory_path):
        if filename.endswith(".json"):  # Ensures only JSON files are read
            file_path = os.path.join(directory_path, filename)
            with open(file_path, 'r', encoding='utf-8') as file:
                # Load JSON content and add it to the list
                all_jsons.append(json.load(file))
    return all_jsons

# Replace 'your_directory_path' with the path to your directory containing JSON files
directory_path = '.'
jsons_list = read_jsons_from_directory(directory_path)

# You can now work with 'jsons_list' which contains all the JSON data
print(f"Loaded {len(jsons_list)} JSON files.")


Loaded 40 JSON files.


In [3]:
import pandas as pd

# Assuming jsons_list is already populated with the JSON objects
data_for_df = []

# Extracting the relevant information from each JSON object
for game in jsons_list:
    prompt_version = game["disposition"]["prompt_version"]
    end_condition = game["metrics"]["end_condition"]
    data_for_df.append({"prompt_version": prompt_version, "end_condition": end_condition})

# Creating a DataFrame
df = pd.DataFrame(data_for_df)

# Aggregating to count different end_conditions for each prompt_version
result_df = df.groupby(['prompt_version', 'end_condition']).size().unstack(fill_value=0)

print(result_df)



end_condition                  EE  RLE  SPYLOOSES  SPYWINS
prompt_version                                            
baseline                        0   10          0        0
remember_bracket_format         0    1          4        5
remember_json_guessing_format   7    1          2        0
remember_sentence_format        0    2          3        5


In [1]:
import os
import json

def parse_messages_and_export(directory_path):
    # Iterate through each file in the directory
    for filename in os.listdir(directory_path):
        if filename.endswith(".json"):
            file_path = os.path.join(directory_path, filename)
            
            # Read JSON file
            with open(file_path, 'r', encoding='utf-8') as file:
                data = json.load(file)
            
            # Extract messages and format them
            formatted_messages = ""
            for message in data["messages"]:
                agent_name = message["agent_name"]
                content = message["content"]
                visible_to = message.get("visible_to", "all")  # Default to 'all' if not specified
                if not isinstance(visible_to, str):  # Convert list to comma-separated string if necessary
                    visible_to = ", ".join(visible_to)
                formatted_messages += f"{agent_name} [{visible_to}]: '{content}'\n\n"
            
            # Define the output text file name (same as JSON but with .txt extension)
            output_filename = os.path.splitext(filename)[0] + ".txt"
            output_file_path = os.path.join(directory_path, output_filename)
            
            # Write the formatted messages to a text file
            with open(output_file_path, 'w', encoding='utf-8') as outfile:
                outfile.write(formatted_messages)
            
            print(f"Exported messages to {output_file_path}")

# Replace 'your_directory_path' with the path to your directory containing JSON files
directory_path = '.'
parse_messages_and_export(directory_path)


Exported messages to ./spyfall_remember_json_guessing_format_2024_03_30_15_17_59.txt
Exported messages to ./spyfall_baseline_2024_03_30_15_22_21.txt
Exported messages to ./spyfall_baseline_2024_03_29_16_13_57.txt
Exported messages to ./spyfall_remember_bracket_format_2024_03_30_10_51_58.txt
Exported messages to ./spyfall_remember_json_guessing_format_2024_03_29_16_24_53.txt
Exported messages to ./spyfall_baseline_2024_03_29_16_12_37.txt
Exported messages to ./spyfall_remember_json_guessing_format_2024_03_29_16_25_48.txt
Exported messages to ./spyfall_remember_sentence_format_2024_03_29_16_48_56.txt
Exported messages to ./spyfall_remember_json_guessing_format_2024_03_29_16_22_51.txt
Exported messages to ./spyfall_remember_sentence_format_2024_03_29_16_41_06.txt
Exported messages to ./spyfall_baseline_2024_03_29_16_14_23.txt
Exported messages to ./spyfall_remember_json_guessing_format_2024_03_29_16_19_24.txt
Exported messages to ./spyfall_remember_sentence_format_2024_03_29_16_48_10.txt


In [5]:
import os
import json
import pandas as pd

def count_files_with_word_mentions(directory_path):
    # Initialize a dictionary to hold the count of files mentioning the words for each prompt_version
    prompt_version_counts = {}

    for filename in os.listdir(directory_path):
        if filename.endswith(".json"):
            file_path = os.path.join(directory_path, filename)

            with open(file_path, 'r', encoding='utf-8') as file:
                game_data = json.load(file)
                prompt_version = game_data["disposition"]["prompt_version"]
                words = []

                # Extract words
                for message in game_data["messages"]:
                    content = message["content"]
                    if content and content.startswith("Your word is:"):
                        word = content.split(": ")[1].split(".")[0]
                        words.append(word)
                
                # Check for mentions of these words in other messages
                mentioned = False
                for message in game_data["messages"]:
                    content = message["content"]
                    if content:
                        for word in words:
                            if word in content and not content.startswith("Your word is:"):
                                mentioned = True
                                break
                    if mentioned:
                        break

                if mentioned:
                    if prompt_version in prompt_version_counts:
                        prompt_version_counts[prompt_version] += 1
                    else:
                        prompt_version_counts[prompt_version] = 1

    # Convert the dictionary to a DataFrame for easy handling and visualization
    df_counts = pd.DataFrame(list(prompt_version_counts.items()), columns=['Prompt Version', 'Number of Files Mentioning Words'])

    return df_counts

# Replace 'your_directory_path' with the path to your directory containing JSON files
directory_path = '.'
df_file_mentions = count_files_with_word_mentions(directory_path)

# Display the DataFrame
print(df_file_mentions)


             Prompt Version  Number of Files Mentioning Words
0  remember_sentence_format                                 4
1   remember_bracket_format                                 2


In [6]:
import os
import json
import pandas as pd

def calculate_average_describing_stage_mentions(directory_path):
    # Data structure to hold counts of the specific message per prompt_version
    data = []

    for filename in os.listdir(directory_path):
        if filename.endswith(".json"):
            file_path = os.path.join(directory_path, filename)

            with open(file_path, 'r', encoding='utf-8') as file:
                game_data = json.load(file)
                prompt_version = game_data["disposition"]["prompt_version"]

                # Count occurrences of the specific message in this file, ensuring content is not None
                message_count = sum(1 for message in game_data["messages"]
                                    if message["content"] and message["content"].startswith("Host: Now it's the describing stage"))

                data.append({"prompt_version": prompt_version, "message_count": message_count})

    # Convert data to a DataFrame
    df = pd.DataFrame(data)

    # Group by prompt_version and calculate the average message count
    result_df = df.groupby('prompt_version')['message_count'].mean().reset_index(name='average_count')

    return result_df

# Replace 'your_directory_path' with the path to your directory containing JSON files
directory_path = '.'
avg_describing_stage_mentions = calculate_average_describing_stage_mentions(directory_path)

# Display the result
print(avg_describing_stage_mentions)


                  prompt_version  average_count
0                       baseline            1.0
1        remember_bracket_format            3.7
2  remember_json_guessing_format            1.5
3       remember_sentence_format            3.6
